In [8]:
# IMPORTS
import math
from collections import Counter
from functools import reduce
from string import punctuation
from typing import Optional

from nltk import sent_tokenize

# Unsmoothed n-grams

To start, you will write a program that computes unsmoothed unigram and bigram probabilities from
the training corpus.

You are given a tokenized opinion spam corpus as input. You may want to do additional preprocessing, based on the design decisions you make.

You may use existing tools just for the purpose of preprocessing, but you must write the code for gathering n-gram counts and computing n-gram probabilities yourself.

For example, consider the simple corpus consisting of the sole sentence: 

`the students like the assignment`

Part of what your program would compute for a unigram and bigram model, for example, would be the
following:

```
P (the) = 0.4, P (like) = 0.2
P (the|like) = 1.0, P (students|the) = 0.5
```

**Preprocessing**: The files included are already tokenized and hence it should be straightforward to
obtain the tokens by using space as the delimiter. Feel free to do any other preprocessing that you might
think is important for this corpus.

In [9]:
# Read in train set
with open('./A1_DATASET/train.txt', 'r') as f:
    train_text = f.read().splitlines()

# Split train set into sentences
train_sents = []
for line in train_text:
    train_sents += sent_tokenize(line)

In [10]:
# Example train sentence
train_sents[3]

'I am looking at a brick wall , and getting no sleep .'

In [11]:
def preprocess(text: str) -> str:
    """
    Preprocesses the text by doing the following:
    - Removt punctuation
    - Convert to lowercase
    """
    text = text.translate(str.maketrans('', '', punctuation))
    text = text.lower()
    
    return text

In [12]:
train = [preprocess(text).split() for text in train_sents]

In [13]:
# Example preprocessed train sentence
print(train[3])

['i', 'am', 'looking', 'at', 'a', 'brick', 'wall', 'and', 'getting', 'no', 'sleep']


In [14]:
def ngrams(text: list, n: int) -> list[tuple]:
    """
    Returns a list of n-grams from the text
    Includes start and end tokens
    """
    start_tokens = n - 1 if n > 1 else 1
    text = ['<s>'] * start_tokens + text + ['</s>']
    return [tuple(text[i:i+n]) for i in range(len(text)-n+1)]

In [15]:
# Compute unigrams
train_unigrams = [ngrams(sent, 1) for sent in train]
train_unigram_counts = reduce(lambda x, y: Counter(x) + Counter(y), train_unigrams)

train_boundary_token_count = train_unigram_counts[("<s>",)] + train_unigram_counts[("</s>",)]
train_total_tokens = sum(train_unigram_counts.values()) - train_boundary_token_count # Subtract boundary token counts from total token count
train_vocab_size = len(train_unigram_counts) - 2 # Subtract boundary tokens from vocab size
train_unigram_probs = {k: v / train_total_tokens for k, v in train_unigram_counts.items() if k not in [("<s>",), ("</s>",)]}

In [16]:
# Display top 10 unigram counts
train_unigram_counts.most_common(10)

[(('<s>',), 5406),
 (('</s>',), 5406),
 (('the',), 5302),
 (('and',), 2593),
 (('a',), 2247),
 (('to',), 2090),
 (('was',), 1826),
 (('i',), 1711),
 (('in',), 1260),
 (('we',), 1117)]

In [17]:
# Display top 10 unigram probabilities
sorted(train_unigram_probs.items(), key=lambda x: x[1], reverse=True)[:10]

[(('the',), 0.06677665965566316),
 (('and',), 0.032657842038312825),
 (('a',), 0.028300104535321603),
 (('to',), 0.026322749656796686),
 (('was',), 0.022997770752780262),
 (('i',), 0.02154938979080341),
 (('in',), 0.01586921749644202),
 (('we',), 0.014068187256766458),
 (('of',), 0.01317396944545901),
 (('hotel',), 0.013048023274852327)]

In [18]:
# Compute bigrams
train_bigrams = [ngrams(sent, 2) for sent in train]
train_bigram_counts = reduce(lambda x, y: Counter(x) + Counter(y), train_bigrams)
train_bigram_probs = {k: v / train_unigram_counts[(k[0],)] for k, v in train_bigram_counts.items()}

In [19]:
# Display top 10 bigram counts
train_bigram_counts.most_common(10)

[(('<s>', 'the'), 991),
 (('<s>', 'i'), 710),
 (('<s>', 'we'), 462),
 (('the', 'hotel'), 414),
 (('in', 'the'), 412),
 (('of', 'the'), 343),
 (('at', 'the'), 333),
 (('the', 'room'), 295),
 (('and', 'the'), 281),
 (('to', 'the'), 264)]

In [20]:
# Pack unigram and bigram counts into a dictionary
train_ngram_counts = {
    1: train_unigram_counts,
    2: train_bigram_counts
}

# Smoothing and unknown words

Firstly, you should implement at least one method to handle unknown words. 

Then, you will need to implement two smoothing methods (e.g., Laplace, Add-k smoothing) with different values of k. 

Teams can choose any method(s) they prefer for each. The report should clearly state the selected methods, providing a description for any non-standard approach (e.g., an approach not covered in class).

In [21]:
# Replace words with <unk> if they appear less than or equal to "n" times
n = 1
unk_words = {k[0] for k, v in train_unigram_counts.items() if v <= n}

# Replace decided unk_words with <UNK> in train set
unk_train = [
    ["<UNK>" if word in unk_words else word for word in sent] for sent in train
]

In [22]:
# Example sentence with <UNK> tokens
for sent in unk_train:
    if "<UNK>" in sent:
        print(sent)
        break

['when', 'speaking', 'to', 'the', 'front', 'desk', 'i', 'was', 'told', 'that', 'they', 'were', 'simply', '<UNK>', 'my', 'request', 'for', 'an', 'upper', 'floor', 'which', 'i', 'had', 'requested', 'for', 'a', 'better', 'view']


In [23]:
# Compute unigrams
train_unk_unigrams = [ngrams(sent, 1) for sent in unk_train]
train_unk_unigram_counts = reduce(lambda x, y: Counter(x) + Counter(y), train_unk_unigrams)
train_unk_vocab_size = len(train_unk_unigram_counts) - 2 # Subtract boundary tokens from vocab size
train_unk_unigram_probs = {k: v / train_total_tokens for k, v in train_unk_unigram_counts.items() if k not in [("<s>",), ("</s>",)]}

In [24]:
# Display top 10 unigram counts
train_unk_unigram_counts.most_common(10)

[(('<s>',), 5406),
 (('</s>',), 5406),
 (('the',), 5302),
 (('<UNK>',), 3113),
 (('and',), 2593),
 (('a',), 2247),
 (('to',), 2090),
 (('was',), 1826),
 (('i',), 1711),
 (('in',), 1260)]

In [25]:
# Display top 10 unigram probabilities
sorted(train_unk_unigram_probs.items(), key=lambda x: x[1], reverse=True)[:10]

[(('the',), 0.06677665965566316),
 (('<UNK>',), 0.039207042909860323),
 (('and',), 0.032657842038312825),
 (('a',), 0.028300104535321603),
 (('to',), 0.026322749656796686),
 (('was',), 0.022997770752780262),
 (('i',), 0.02154938979080341),
 (('in',), 0.01586921749644202),
 (('we',), 0.014068187256766458),
 (('of',), 0.01317396944545901)]

In [26]:
# Compute bigrams
train_unk_bigrams = [ngrams(sent, 2) for sent in unk_train]
train_unk_bigram_counts = reduce(lambda x, y: Counter(x) + Counter(y), train_unk_bigrams)
train_unk_bigram_probs = {k: v / train_unk_unigram_counts[(k[0],)] for k, v in train_unk_bigram_counts.items()}

In [27]:
# Display top 10 bigram counts
train_unk_bigram_counts.most_common(10)

[(('<s>', 'the'), 991),
 (('<s>', 'i'), 710),
 (('<s>', 'we'), 462),
 (('<UNK>', '</s>'), 443),
 (('the', 'hotel'), 414),
 (('in', 'the'), 412),
 (('of', 'the'), 343),
 (('at', 'the'), 333),
 (('the', 'room'), 295),
 (('and', 'the'), 281)]

In [53]:
def laplace_smoothing(ngram: tuple[str, ...], ngram_counts: dict[int, dict], vocab_size: int, total_tokens: Optional[int] = None) -> float:
    """
    Returns the Laplace smoothed probability for a given ngram
    """
    n = len(ngram)
    if n == 1:
        return (ngram_counts[1][ngram] + 1) / (total_tokens + vocab_size)
    else:
        return (ngram_counts[n][ngram] + 1) / (ngram_counts[n-1][ngram[:-1]] + vocab_size)

In [54]:
def add_k_smoothing(ngram: tuple[str, ...], ngram_counts: dict[int, dict], vocab_size: int, total_tokens: Optional[int] = None, k: int = 1) -> float:
    """
    Returns the add-k smoothed probability for a given ngram
    """
    n = len(ngram)
    if n == 1:
        return (ngram_counts[1][ngram] + k) / (total_tokens + k * vocab_size)
    else:
        return (ngram_counts[n][ngram] + k) / (ngram_counts[n-1][ngram[:-1]] + k * vocab_size)

# Perplexity

Implement code to compute the perplexity of a `development set` (*A `development set` is just another way to refer to the validation set—part of a dataset distinct from the training portion)*. Compute and report the perplexity of your model (with variations) on it. Compute perplexity as follows:

![](https://i.gyazo.com/4069027ef1730b89862b96cbb1526b7e.png)

Where:

- N is the total number of tokens in the test corpus.
- P(w_i|w_i1, ..., w_in+1) is the n-gram probability of your model.

Under the second definition above, perplexity is a function of the average (per-word) log probability. Use this to avoid numerical computation errors.

If you experimented with more than one type of smoothing and unknown word handling, you should report and compare the perplexity results of experiments among some of them.

In [30]:
# Read in test set
with open('./A1_DATASET/val.txt', 'r') as f:
    val_text = f.read().splitlines()
    
# Split test set into sentences
val_sents = []
for line in val_text:
    val_sents += sent_tokenize(line)

In [31]:
val = [preprocess(text).split() for text in val_sents]

In [32]:
# Example preprocessed val sentence
print(val[3])

['the', 'staff', 'was', 'friendly', 'and', 'the', 'fitness', 'center', 'while', 'not', 'huge', 'was', 'wellequipped', 'and', 'clean']


In [33]:
val_unigrams = [ngrams(sent, 1) for sent in val]
val_bigrams = [ngrams(sent, 2) for sent in val]

In [34]:
def perplexity(sent_probs: list[float]) -> float:
    """
    Returns the perplexity of the test dataset
    """
    l = (1 / len(sent_probs)) * sum([math.log2(prob) for prob in sent_probs])
    return 2 ** -l

**Perplexity For Train Unigrams**

4 Methods:
- Default model
- \<UNK\> words
- Laplace Smoothing
- Add-K Smoothing
    - k=0.5
    - k=0.05
    - k=0.01

In [35]:
# Default model

In [36]:
# Using <UNK> tokens


In [37]:
# Using Laplace Smoothing

In [38]:
# Using Add-k Smoothing

**Perplexity For Train Bigrams**

4 Methods:
- Default model
- \<UNK\> words
- Laplace Smoothing
- Add-K Smoothing
    - k=0.5
    - k=0.05
    - k=0.01

In [39]:
# Default model

In [40]:
# Using <UNK> tokens


In [41]:
# Using Laplace Smoothing

In [42]:
# Using Add-k Smoothing

**Perplexity For Validation Unigrams**

3 Methods:
- \<UNK\> words
- Laplace Smoothing
- Add-K Smoothing
    - k=0.5
    - k=0.05
    - k=0.01

In [60]:
# Using <UNK> tokens
val_sent_probs_unk = [
    math.prod(
        train_unk_unigram_probs.get(unigram, train_unk_unigram_probs[("<UNK>",)])
        for unigram in sent_unigrams
        if unigram not in [("<s>",), ("</s>",)]
    )
    for sent_unigrams in val_unigrams
]

print('(Perplexity | Unigrams | <UNK> Tokens):', perplexity(val_sent_probs_unk))

(Perplexity | Unigrams | <UNK> Tokens): 1.2658373379148482e+37


In [61]:
# Using Laplace Smoothing
val_sent_probs_lp = [
    math.prod(
        laplace_smoothing(ngram=unigram, ngram_counts=train_ngram_counts, vocab_size=train_vocab_size, total_tokens=train_total_tokens)
        for unigram in sent_unigrams
        if unigram not in [("<s>",), ("</s>",)]
    )
    for sent_unigrams in val_unigrams
]

print('(Perplexity | Unigrams | Laplace Smoothing):', perplexity(val_sent_probs_lp))

(Perplexity | Unigrams | Laplace Smoothing): 2.1938464045883443e+40


In [69]:
# Using Add-k Smoothing
k = [0.5, 0.05, 0.01]
for k_val in k:
    val_sent_probs_k = [
        math.prod(
            add_k_smoothing(ngram=unigram, ngram_counts=train_ngram_counts, vocab_size=train_vocab_size, total_tokens=train_total_tokens, k=k_val)
            for unigram in sent_unigrams
            if unigram not in [("<s>",), ("</s>",)]
        )
        for sent_unigrams in val_unigrams
    ]
    print(f'(Perplexity | Unigrams | Add-K Smoothing | k={k_val}):', perplexity(val_sent_probs_k))

(Perplexity | Unigrams | Add-K Smoothing | k=0.5): 2.4938717985987523e+40
(Perplexity | Unigrams | Add-K Smoothing | k=0.05): 7.02056030501304e+40
(Perplexity | Unigrams | Add-K Smoothing | k=0.01): 1.6200249907672142e+41


**Perplexity For Validation Bigrams**

3 Methods:
- \<UNK\> words
- Laplace Smoothing
- Add-K Smoothing
    - k=0.5
    - k=0.05
    - k=0.01

In [47]:
# Using <UNK> tokens
val_sent_probs_unk = [
    math.prod(
        train_unk_unigram_probs.get(bigram, train_unk_unigram_probs[("<UNK>",)])
        for bigram in sent_bigrams
    )
    for sent_bigrams in val_bigrams
]

for sent_bigrams in val_bigrams:
    for bigram in sent_bigrams:
        unk_bigram = tuple('<UNK>' if word in unk_words else word for word in bigram)
        train_unk_bigram_probs[unk_bigram] = 

print('(Perplexity | Unigrams | <UNK> Tokens):', perplexity(val_sent_probs_unk))

In [48]:
# Using Laplace Smoothing


In [49]:
# Using Add-k Smoothing


In [50]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

train_sentences = ['an apple', 'an orange']
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in train_sentences]
n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, tokenized_text)
model = MLE(n)
model.fit(train_data, padded_vocab)

test_sentences = ['an apple', 'an ant']
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in test_sentences]

test_data, _ = padded_everygram_pipeline(n, tokenized_text)
for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),model.score(ngram[-1], ngram[:-1])) for ngram in test])

test_data, _ = padded_everygram_pipeline(n, tokenized_text)

for i, test in enumerate(test_data):
  print("PP({0}):{1}".format(test_sentences[i], model.perplexity(test)))

MLE Estimates: [(('an', ()), 0.5), (('apple', ()), 0.25)]
MLE Estimates: [(('an', ()), 0.5), (('ant', ()), 0.0)]
PP(an apple):2.8284271247461903
PP(an ant):inf


In [51]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm import Vocabulary

train_sentences = ['an apple', 'an orange']
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) for sent in train_sentences]

n = 2
train_data = [nltk.bigrams(t,  pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>") for t in tokenized_text]
words = [word for sent in tokenized_text for word in sent]
words.extend(["<s>", "</s>"])
padded_vocab = Vocabulary(words)
model = MLE(n)
model.fit(train_data, padded_vocab)

test_sentences = ['an apple', 'an ant']
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) for sent in test_sentences]

test_data = [nltk.bigrams(t,  pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>") for t in tokenized_text]
for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),model.score(ngram[-1], ngram[:-1])) for ngram in test])

test_data = [nltk.bigrams(t,  pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>") for t in tokenized_text]
for i, test in enumerate(test_data):
  print("PP({0}):{1}".format(test_sentences[i], model.perplexity(test)))

MLE Estimates: [(('an', ('<s>',)), 1.0), (('apple', ('an',)), 0.5), (('</s>', ('apple',)), 1.0)]
MLE Estimates: [(('an', ('<s>',)), 1.0), (('ant', ('an',)), 0.0), (('</s>', ('ant',)), 0)]
PP(an apple):1.2599210498948732
PP(an ant):inf
